In [2]:
import tensorflow as tf
import keras
import pandas as pd
import matplotlib.pyplot as plt
import scipy
import sklearn
from sklearn.model_selection import train_test_split
import numpy as np
import random


In [3]:
def build_MLP(n_nodes=200,activation_func="relu",learning_rate=0.01,input_Shape=[28,28],output_shape=10,use_dropout=False,dropout_rate=0.2,use_l1=False,use_l2=False):
    model = keras.models.Sequential()
    model.add(keras.layers.Flatten(input_shape=input_Shape))
    model.add(keras.layers.Dense(n_nodes, activation=activation_func))
    if use_dropout==True:
        model.add(keras.layers.Dropout(rate=dropout_rate))
    model.add(keras.layers.Dense(output_shape, activation="softmax"))
    optimizer = keras.optimizers.SGD(learning_rate=learning_rate)
    model.compile(loss="sparse_categorical_crossentropy",optimizer=optimizer, metrics=["accuracy"])
    return model

def build_test_model():
    model = keras.models.Sequential()
    model.add(keras.layers.Flatten(input_shape=[28, 28]))
    model.add(keras.layers.Dense(300, activation="relu"))
    model.add(keras.layers.Dense(100, activation="relu"))
    model.add(keras.layers.Dense(10, activation="softmax"))
    optimizer_sgd=keras.optimizers.SGD(learning_rate=0.01)
    model.compile(loss="sparse_categorical_crossentropy",optimizer=optimizer_sgd, metrics=["accuracy"])
    return model

In [33]:
fashion_mnist=keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()
X_test=X_test/255.0

x_train,x_val,y_train,y_val=sklearn.model_selection.train_test_split(X_train_full/255.0,y_train_full,test_size=0.1,random_state=42)
print(x_train.shape)



(54000, 28, 28)


In [34]:
#hyper parameter tuning through random search
tests=200
n_nodes=np.arange(199)+1
activation_functions=["elu","relu","sigmoid"]
learning_rates=[0.0003,0.03]
used_parameters=[]
gained_scores=[]
early_stoping=keras.callbacks.EarlyStopping(patience=3)
for i in range(tests):
    print("search "+str(i+1))
    nodes=n_nodes[random.randint(0,len(n_nodes)-1)]
    activation=activation_functions[random.randint(0,len(activation_functions)-1)]
    learning_rate=random.uniform(learning_rates[0],learning_rates[1])
    used_parameters.append([nodes,activation,learning_rate])

    model=build_MLP(n_nodes=nodes,activation_func=activation,learning_rate=learning_rate)
    #model=build_test_model()
    history = model.fit(x_train, y_train, epochs=20,validation_data=(x_val, y_val),callbacks=[early_stoping],verbose=0)
    gained_scores.append(model.evaluate(X_test, y_test,verbose=1)[1])
#print(used_parameters)
#print(gained_scores)

results=zip(gained_scores,used_parameters)
three_best=sorted(results,reverse=True)[:3]
print(sorted(results,reverse=True)[:3])


search 1
313/313 [==============================] - 0s 1ms/step - loss: 0.3809 - accuracy: 0.8603
search 2
313/313 [==============================] - 0s 880us/step - loss: 0.5307 - accuracy: 0.8177
search 3
313/313 [==============================] - 0s 909us/step - loss: 0.4613 - accuracy: 0.8382
search 4
313/313 [==============================] - 0s 953us/step - loss: 0.3855 - accuracy: 0.8642
search 5
313/313 [==============================] - 0s 861us/step - loss: 0.4079 - accuracy: 0.8528
search 6
313/313 [==============================] - 0s 924us/step - loss: 0.4202 - accuracy: 0.8464
search 7
313/313 [==============================] - 0s 948us/step - loss: 0.4983 - accuracy: 0.8289
search 8
313/313 [==============================] - 0s 914us/step - loss: 0.3827 - accuracy: 0.8634
search 9
313/313 [==============================] - 0s 935us/step - loss: 0.4522 - accuracy: 0.8398
search 10
313/313 [==============================] - 0s 987us/step - loss: 0.3577 - accuracy: 0.8714
s

In [4]:
try:
    uitkomst=sorted(results,reverse=True)[:3]
except: #backup if you don't want to run randomsearch
    uitkomst=[(0.885200023651123, [197, 'relu', 0.02691421553000728]), (0.8784999847412109, [130, 'relu', 0.028888078594726852]), (0.876800000667572, [112, 'relu', 0.023793749068561666])]

cifar10=keras.datasets.cifar10
(X_train_full_cifar, y_train_full_cifar), (X_test_cifar, y_test_cifar) = cifar10.load_data()
X_test_cifar=X_test_cifar/255.0
print(y_test_cifar.shape) #loof at output shape might be wrong in build_mlp
x_train_cifar,x_val_cifar,y_train_cifar,y_val_cifar=sklearn.model_selection.train_test_split(X_train_full_cifar/255.0,y_train_full_cifar,test_size=0.1,random_state=40)


for i in uitkomst:
    model=build_MLP(n_nodes=i[1][0],activation_func=i[1][1],learning_rate=i[1][2],input_Shape=[32, 32,3])
    history = model.fit(x_train_cifar, y_train_cifar, epochs=20,validation_data=(x_val_cifar, y_val_cifar),callbacks=[early_stoping],verbose=0)
    print(model.evaluate(X_test_cifar, y_test_cifar,verbose=1)[1])

Exception: URL fetch failure on https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz: None -- [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:997)